In [29]:
from kivy.lang import Builder
from kivymd.app import MDApp
from kivymd.uix.screen import Screen
from kivymd.uix.button import MDRectangleFlatButton, MDFlatButton
from kivymd.uix.dialog import MDDialog
import re
from textblob import TextBlob
from tweepy import OAuthHandler
import tweepy


class clean_sentiment:
    def __init__(self, tweet):
        self.tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", tweet).split())

    def get_tweet_sentiment(self):
        analysis = TextBlob(self.tweet)  # create TextBlob object of passed tweet text
        if analysis.sentiment.polarity > 0:  # set sentiment
            self.sentiment = 'positive'
        elif analysis.sentiment.polarity == 0:
            self.sentiment = 'neutral'
        else:
            self.sentiment = 'negative'


class api_cred:
    def __init__(self):  #create your own credential from twitter API
        self.consumer_key = 'VJFZEdGk7cyzbbK#1VPa#1PaAIPgs'
        self.consumer_secret = 'WcA3PZ0Z#1YNuad524X3qUx97ExiEmI0g#1ULZ3JcMMwmxn920xxC7'
        self.access_token = '3257684406-kh5gJuLncLMe9o1#kQVKS60#1FH47XmITXMUgo4qmB'
        self.access_token_secret = 'BPg3GA7pSj86qvFqzmCXPCBGQeaSuPXRKdkP2UC1W85A1'


class main_obj:
    def __init__(self, In_string):
        self.hastag = "#" + In_string

    def fnmain(self):
        cred = api_cred()
        auth = OAuthHandler(cred.consumer_key, cred.consumer_secret)
        auth.set_access_token(cred.access_token, cred.access_token_secret)
        api = tweepy.API(auth, wait_on_rate_limit=True)
        self.tweet_list = tweepy.Cursor(api.search, q=self.hastag).items(100)

    def fnprocess(self):
        tweets = []  # empty list to store parsed tweets
        for tweet in self.tweet_list:  # parsing tweets one by one
            parsed_tweet = {}  # empty dictionary to store required params of a tweet
            parsed_tweet['text'] = tweet.text  # saving text of tweet
            clean_tweet = clean_sentiment(tweet.text)
            clean_tweet.get_tweet_sentiment()
            parsed_tweet['sentiment'] = clean_tweet.sentiment  # saving sentiment of tweet
            tweets.append(parsed_tweet)
        postt = [tt1 for tt1 in tweets if tt1['sentiment'] == 'positive']
        self.postwt = (100 * len(postt) / len(tweets))
        negtt = [tt1 for tt1 in tweets if tt1['sentiment'] == 'negative']
        self.negtwt = (100 * len(negtt) / len(tweets))
        neutt = [tt1 for tt1 in tweets if tt1['sentiment'] == 'neutral']
        self.neutwt = (100 * len(neutt) / len(tweets))

instring = """
MDTextField:
    hint_text: 'Enter Hastag'
    helper_text:"Do not keep it blank | It take some time"
    helper_text_mode:"persistent"
    icon_right:"android"
    icon_right_color:app.theme_cls.primary_color
    pos_hint:{'center_x':0.5,'center_y':0.5}
    size_hint_x:None
    width:300
"""

class MyApp(MDApp):
    def build(self):
        self.screen = Screen()
        self.theme_cls.primary_palette= "Green"
        button = MDRectangleFlatButton(text="SHOW",
                                       pos_hint={'center_x':0.5,'center_y':0.4},
                                       on_release=self.show_data)
        self.msg = Builder.load_string(instring)
        self.screen.add_widget(self.msg)
        self.screen.add_widget(button)
        return self.screen
    def show_data(self,obj):
        in_str = self.msg.text
        if ((in_str != ' ')&(in_str != '')):
            mainobj = main_obj(in_str)
            mainobj.fnmain()
            mainobj.fnprocess()
            self.results = "Positive :"+str(mainobj.postwt)+"%\nNegative :" + str(mainobj.negtwt)+"%\nNeutral  :" + str(mainobj.negtwt)+"%"
            close_button = MDFlatButton(text='Close',on_release=self.close_dialog)
            self.dialog = MDDialog(title="Twittes for : %s"%mainobj.hastag,
                                   text=self.results,
                                   size_hint=(0.7,1),
                                   buttons=[close_button])
            self.dialog.open()
        else:
            close_button = MDFlatButton(text='Close', on_release=self.close_dialog)
            self.dialog = MDDialog(title="Enter valid HASTAG",
                                   size_hint=(0.7, 1),
                                   buttons=[close_button])
            self.dialog.open()
    def close_dialog(self,obj):
        self.dialog.dismiss()


# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    MyApp().run()
